# 04: Go capabilities: Intro to methods, interfaces, and type assertions

This section is a summary of https://go.dev/tour/

## Objectives
+ methods 
+ interfaces
+ type assertions

## Summary

This section teaches you methods and interfaces, the constructs that defines objects and their behaviors.

## Methods

Go does not have classes, but it allows you to define methods on types.

A method is a function with a special *receiver* argument. This *receiver* appears in its own argument list between the `func` keyword and the method name.

In the example below, a `Vertex` type is defined. On it, and `Dist()` method is declared:

```go
type Vertex {
  X, Y float64
}

func (v Vertex) Dist() float64 {
  return math.Sqrt(v.X * v.X + v.Y * v.Y)
}

func main() {
  v := Vertex(3, 4)
  fmt.Println(v.Dist()) // -> 5
}
```

## Lab

Create a Vector2D struct and define a method Length() that returns the length of the vector.

| EXAMPLE: |
| :------- |
| See [01_methods](01_methods/) for a runnable example. |

You can declare a method on non-struct types too.

However, you can only declare a method with a receiver whose type is defined in the same package as the method, which means you cannot define methods on built-in types (such as `int`) because those are defined on other packages.

### Lab

Write a program that defines a type `MyFloat` which is an alias of the `float64` type. Then define a method `MyFloat.Abs()` that returns the absolute value $  | f | $ of a float.

| EXAMPLE: |
| :------- |
| See [02_methods-on-non-structs](02_methods-on-non-structs/) for a runnable example. |

You can declare methods with pointer receivers.

This means the receiver type has the literal syntax `*T` for some type T. Note that T cannot be a pointer (e.g., T cannot be something like `*int`).

Methods with pointer receivers can modify the value which the receiver point to. Since methods often need to modify their receiver, pointer receivers are more common than value receivers.

| NOTE: |
| :---- |
| With a value receiver, a method operate on a copy of the original value (as it happens with value arguments in regular functions). If you want to change the underlying value in methods (or in regular functions) use a pointer. |

### Lab

Write a program that defines a `Vertex` type with fields `X` and `Y` as `float64` values. Then define on it a `Scale` method that scales both coordinates using pointer receivers.
What happens if you remove the pointer?
Define also an `Abs()` function on Vertex that returns the result of $ \sqrt{X^2 + Y^2} $. Is a pointer receiver needed in this case?

| EXAMPLE: |
| :------- |
| See [03_method-pointers](03_method-pointers/) for a runnable example. |

Methods can always be rewritten as standalone functions, and this approach is very common in Go.

### Lab

Rewrite [03_method-pointers](03_method-pointers/) using standalone functions.

| EXAMPLE: |
| :------- |
| See [04_functions-standalone](04_functions-standalone/) for a runnable example. |

If you compare the syntax used for [03_method-pointers](03_method-pointers/) and [04_functions-standalone](04_functions-standalone/) labs, you'll see that functions with a pointer argument require a pointer, while methods won't:

```go
// method
v.Scale(2)    // Scale is defined as func (v *Vertex) Scale(s float64)

Scale(&v, 2)  // Scale is defined as func Scale(v *Vertex, s float64)
```

Note however that methods with a pointer receiver can be applied to pointers too:

```go
v := Vertex{3, 4}
p := &v
p.Scale(10)  // OK
```

There are two reasons to use a pointer receiver:
+ To enable to the method to modify the value its receiver points to.
+ To avoid copying the value on each method call, which will have performance benefits especially on large structs.

| NOTE: |
| :---- |
| In general, all methods on a given type should have either value or pointer receivers, but not a mixture of both. |

### Lab

Repeat [03_method-pointers](03_method-pointers/) but defining all the method with pointer receivers.

| EXAMPLE: |
| :------- |
| See [05_methods-pointer-receives](05_methods-pointer-receivers/) for a runnable example. |

## Interfaces

An interface type is defines as set of method signatures.

A value of a given interface type can hold any value that implements those methods.

You can define an interface using the following syntax:

```go
type Abser interface {
  Abs() float64
}
```

Through an interface value you can call the corresponding method:

```go
var a Abser
v := Vertex{3, 4}
a = &v
a.Abs() // -> will call func (v Vertex) Abs() float 64
```

### Lab

Write a program that defines an interface `Abser` for an `Abs()` method that takes no parameters and returns a `float64`.

Then, define a `MyFloat` type which wraps a `float64` and a `Vertex` type which is a struct with `X` and `Y` fields, boths `float64`.

Then define an `Abs()` method for `MyFloat` that calculates the absolute value of the float, and an `Abs()` method for a `*Vertex` that returns $ \sqrt{x^2 + y^2} $.

Then in main, define one value for `MyFloat` named `f` (`-math.Sqrt2`) and one value for a `Vertex` initialized to `(3, 4)` named `v`.

Then define a value for the `Abser` interface named `a` and do the following assignments:

```go
a = f
a = &v
```

Why does that assignment work?

Then do:

```go
a = v
```

Why does that assignment fails?

| EXAMPLE: |
| :------- |
| See [06_interfaces-hello](06_interfaces-hello/) for a runnable example. |


In Go, interfaces are implemented implicitly &mdash; there is no "implements" keyword in Go.

Implicit interfaces decouple the definition of an interface from its implementation, which could then appear in any package without prearrangement.

| NOTE: |
| :---- |
| Remember that methods can only be defined in the same package as the type in which it is defined. |

### Lab

Define an interface `I` that declares a function `M()`. Then define a struct `T` that contains a string `S` as its field.
Then define a method `M()` on the struct that prints the content of `S`. In the `main()` function, define a value of the interface and initialize it to a value of `T`. Confirm that you can call `M()`.

| EXAMPLE: |
| :------- |
| See [07_interfaces-implicit-impl](07_interfaces-implicit-impl/) for a runnable example. |

Under the hood, interface values can be thought of as a tuple of a value and a concrete type:

```
(value, type)
```

That is, an interface value is tightly associated to a value of a concrete type.

As a result, calling a method on an interface value executed the method of the same name on its underlying type.

### Lab

Write a program that defines an interface `I` with an `M()` method. Then define a struct `T` with a field `S` of type string. Define a method on `*T` called `M()` that prints the content of the string. Then define a type `F` that aliases a `float64` and define a method `M()` on `F` that prints the float. In main, declare a value of the interface and validate that you can use it to call `M()` on both types.
Additionally, define a function `describe(i I)` that prints the tuple `(value, type)`. HINT: you can use the format specifier `%T` to print the type of a value.

| EXAMPLE: |
| :------- |
| See [08_interfaces-interface-values](08_interfaces-interface-value/) for a runnable example. |

If the concrete value inside the interface itself is ´nil`, the method will be called with a nil receiver.

While this would trigger a null pointer exception in some languages, in Go it is common to write methods that gracefully handle being called with a nil receiver.

Note that an interface value that holds a nil concrete value is itself *non-nil*.

### Labs

Write a program the defines an interface `I` with a method `M()`, and a struct `T` with a string field `S`. Then define an implementation of the `M()` method on `*T` that includes a guard to gracefully handle the situation that it is called with a `nil` argument by printing `"<nil>"`. In the main function, create code that checks that the `M()` method works well on nin and non-nil concrete types. Additionally, define a function `describe(i I)` that prints the tuple `(value, type)` and use it to get instrospection on both cases.

| EXAMPLE: |
| :------- |
| See [09_interfaces-interface-nil-value](09_interfaces-interface-nil-value/) for a runnable example. |

Calling a method on a nil interface value (one that has not been assigned to any concrete type) is a run-time error because there is no type inside the interface type to indicate which concrete method should be called.

Those run-time errors will look something like:

```
panic: runtime error: invalid memory address or nil pointer dereference
[signal SIGSEGV: segmentation violation code=0x1 addr=0x0 pc=0x481ce1]
```

Calling a method that doesn't include the necessary guardrails on a non-nil interface that holds a nil concrete value won't trigger a run-time error. It will use the zero-value of the type instead.


### Lab

Modify [09_interfaces-interface-nil-value](09_interfaces-interface-nil-value/) code to illustrate that you get a run-time errors when you call a method on a nil interface, and when you call a method on a non-nil interface that holds a nil value, and that method hasn't taken the necessary precautions to handle the problem gracefully.

| EXAMPLE: |
| :------- |
| See [10_interface-nil-interface-values](10_interface-nil-interface-values/) for a runnable example. |


The empty interface is an interface that specifies zeero methods in its definition:

```go
interface{}
```

An empty interface may hold values of any type, because every type implements at least zero methods.

This construct is used by code that handles values of unknown type. For example, `fmt.Print()` takes arguments of type `interface{}` because it cannot predict the type of arguments it'll receive.

### Lab

Write a program the declares a function `describe()` that can take an argument of any kind. The function should print the tuple `(value, type)` for any argument it receives.

| EXAMPLE: |
| :------- |
| See [11_interface-empty-interface](11_interface-empty-interface/) for a runnable example. |

## Type assertions

A type assertion provides access to an interface value's underlying concrete value:

```go
// asserts that the interface value `i` holds
// the concrete type T and assigns the underlying
// T value to the variable t
t := i.(T)
```

If `i` does not hold a `T`, the statement will trigger a panic.


In order to test whether an interface value holds a specific type, a type assertion can return two values: the underlying value and a boolean value indicating whether the assertion succeeded:

```go
// if ok == true, t will hold the T value
// otherwise, the t will be the zero value
t, ok := i.(T)
```


| NOTE: |
| :---- |
| Note the similarity between the expression above and the one used when checking if a given key is available in a map `val, ok := myMap[key]`. |

### Lab

Write a program that defines an interface value `i` of type `interface{}` and value `"Hello"`.
Then, in the program, check that you can access the underlying string value from the interface using type assertion, and also, that when testing the assertion, the value of `ok` is true.

Finally, use the test assertion syntax to check if you can read the underlying value as a `float64` and how a panic is triggered when you try to do directly the assertion.

| EXAMPLE: |
| :------- |
| See [12_type_assertions-hello](12_type_assertions-hello/) for a runnable example. |

## Type switches

A type switch is a construct that permits several type assertions in series.

The syntax is similar to a regular switch statement, but the cases are compared against the type of value held and not the value itself:

```go
switch v := i.(type) {
  case T:
    // logic to apply when i holds a T value
  case S:
    // logic to apply when i holds an S value
  default:
    // no match, v has the same type as i
}
```

Note that `type` is a keyword, and it replaces the specific type used in type assertions.

### Lab

Write a program that defines a function `do()` that receives an argument `i` of type `interface{}`.
In the function body, implement a type switch to detect whether `i` holds an `int`, a `string`, or something different from an `int` or a `string`.
+ When it is an int, use the underlying value to multiply it by 2 and print it.
+ When it is a string, calculate its length and print it using quotes (HINT: use `%q`).
+ In any other case, just type the underlying value (HINT: use `%T`)

Then in the main function try the three cases.

| EXAMPLE: |
| :------- |
| See [13_type_switches-hello](13_type_switches-hello/) for a runnable example. |

## Stringers

One of the most ubiquitous interfaces is `Stringer` defined in the `fmt` package:

```go
type Stringer interface {
  String() string
}
```

A `Stringer` is a type that can describe itself as a string.

Many packages look for this interface to print values, so you should implement it in your types to make well behave with `fmt` packages and others.

For example, when implementing that method in one of your types, your `String()` implementation will be called when `fmt.Println()` is called.

### Lab

Create a struct type `Person` defined by a string and an int identifying the person's name and age.

Then, implement the `Stringer` interface in the struct with a function that reports the person's name and age in the format `name (age)`.

In the main function, invoke `fmt.Println()` on a Person value. What happens if you don't implement the `Stringer()` interface?

| EXAMPLE: |
| :------- |
| See [14_stringer-hello](14_stringer-hello/) for a runnable example. |

### Lab

Write a program that makes the `IPAddr` type defined as below implement `fmt.Stringer` to print the content of an `IPAddr` value shown as a dotted quad (x.y.z.w).

```go
type IPAddr [4]byte
```

| EXAMPLE: |
| :------- |
| See [15_stringer-ipaddr](15_stringer-ipaddr/) for a runnable example. |

## Errors

Go programs express error state with `error` interface values.

The `error` type is a built-in interface similar to `fmt.Stringer`:

```go
type error interface {
  Error() string
}
```

Many packages look for the `error` interface when dealing with errors, like `fmt` does for printing errors.

Functions often return an `error` value, and your functions should handle errors by testing whether that error equals `nil`:

```go
i, err := strconv.Atoi("42")
if err != nil {
  fmt.Printf("Couldn't convert number: %v\n", err)
  return
}
fmt.Println("Converted integer:", i)
```

### Lab

Write a program that defines a `MyError` struct with fields `When` of type `time.Time` and `What` of type `string`.
Then define an implementation of the `Error()` function of the `error` interface that returns the error string as "at when, what".
Also define a `run()` function that returns a pointer to an instance of &MyError.

Finally, invoke the `run()` function and print the error if not nil.

HINT: you can do very succinctly:

```go
if err := run(); err != nil {
  fmt.Println(err)
}
```

| EXAMPLE: |
| :------- |
| See [16_errors-hello](16_errors-hello/) for a runnable example. |

### Lab

Using the `Sqrt()` function from [06_loops-and-funcs](../05_go_tour_002/06_loops-and-funcs/sqrt-loops-and-funcs/) modify it so that it returns an `error` value.

`Sqrt` should return a non-nil error value when given a negative number, as roots of negative numbers are not supported by the algorithm.

In order to so so, create a new type `type ErrNegativeSqrt float64` and make it an error by implementing the `error` interface such that `ErrNegativeSqrt(-2).Error()` returns the message `"cannot Sqrt negative number: -2"`.

| NOTE: |
| :---- |
| A call to `fmt.Sprint(e)` inside the `Error()` method will send the program into an infinite loop. Can you explain why?<br>It can be avoided by converting `e` first using `fmt.Sprint(float64(e))`. |

Change your `Sqrt` function to return an `ErrNegativeSqrt` value when given a negative number.

| EXAMPLE: |
| :------- |
| See [17_errors-sqrt](17_errors-sqrt/) for a runnable example. |

## Readers

The `io` package specifies the `io.Reader` interface, which represents the read end of a stream of data.

The Go standard library contains many implementations of this interface, including files, network connections, compressors, ciphers...

The `io.Reader` interface has a `Read` method:

```go
func (T) Read(b []byte) (n int, err error)
```

`Read()` populates the given byte slice with data and returns the number of bytes populated and an error value. It returns an `io.EOF` error when the stream ends.

### Lab

Write a program that creates a `strings.Reader`, an implementation of the `io.Reader` interface, that is backed by a string using `strings.NewReader()`.
Then define a byte slice of length 8, and read from the Reader, consuming 8 bytes at a time, until the stream is exhausted.

While reading, print two lines for every read operation:
+ The first one must show the values returned by `Read()`, as well as the byte slice contents.
+ The second should print the contents of the slice from the beginning to the number of bytes returned by the `Read()` operation.

| EXAMPLE: |
| :------- |
| See [18_reader-hello](18_reader-hello/) for a runnable example. |

### Lab

Implement a `Reader` type that emits an infinite stream of ASCII character `'A'`.

| EXAMPLE: |
| :------- |
| See [19_reader-custom-reader](19_reader-custom-reader/) for a runnable example. |

### Lab

A common pattern is an `io.Reader` that wraps another `io.Reader` modifying the stream in some way.

For example, the `gzip.NewReader()` function takes an `io.Reader` that is a stream of compressed data and returns a `*gzip.Reader` which is another type that implements `io.Reader` but represents the stream of decompressed data.

Implement a `rot13Reader` that implements `ioReader` and reads from an `ioReader` modifying the stream by applying the [rot13](https://en.wikipedia.org/wiki/ROT13) to all alphabetical characters.

The `rot13Reader` struct should be defined as:

```go
type rot13Reader struct {
  r io.Reader
}
```

| EXAMPLE: |
| :------- |
| See [20_reader-rot-reader] for a runnable example. |

## Images

The package `Image` defines the `Image` interface:

```go
type Image interface {
  ColorModel()  color.Model
  Bounds()      Rectangle
  At(x, y int)  color.Color
}
```

The `color.Color` and `color.Model` are also interfaces, but the following predefined implementations are available `color.RGBA` and `color.RGBAModel`. The `Rectangle` is an `image.Rectangle` that is defined by the four coordinates of the top left and bottom right vertices.

### Lab

Write a program that creates a new RGBA image for the Rectangle with top-left vertex (0, 0) and bottom-right vertex (100, 100) using the function `image.newRGBA()`.

Then validate it implements the interface by accessing the `Bounds()` and `At` functions.

| EXAMPLE: |
| :------- |
| See [21_image-hello](21_image-hello/) for a runnable example. |

### Lab

Let's write an image generator similar to the one created in [pic-slice](../06_go_tour_003/15_pic-slice/pic-slice) that returns an implementation of `image.Image` instead of 2D matrix (2D slice).

Define your own `Image` type, and implement the necessary methods, and call `pic.ShowImage`. `Bounds` should return an `image.Rectangle` like `image.Rect(0, 0, w, h)`, and `ColorModel` should return `color.RGBAModel`. `At` should return a color; the value `v` in the previous implementation of the last picture generator corresponds to the `color.RGBA{v, v, 255, 255}`.

| EXAMPLE: |
| :------- |
| See [22_image-pic-generator](22_image-pic-generator/) for a runnable example. |